In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

CSV_PATH = 'movies.csv'

df = pd.read_csv(CSV_PATH)


# We filter movies after year 2000
filtered_movies_after_year_2000 = df[df['year'] >= 2000]
# We drop NaN rows
filtered_movies_after_year_2000 = filtered_movies_after_year_2000.dropna()
# Add profitability column
filtered_movies_after_year_2000['profitability'] = filtered_movies_after_year_2000['gross'] - filtered_movies_after_year_2000['budget']



score_gross = filtered_movies_after_year_2000[['score','gross','profitability']]
plt.scatter(score_gross['score'],       # x data is the years of education
            score_gross['profitability'],       # y data is the hourly wages
            c='r', alpha=0.3)   # Color is blue, opacity is 0.3     

#add linear regression line to scatterplot 
m, b = np.polyfit(score_gross['score'],score_gross['profitability'], 1)

#add linear regression line to scatterplot 
plt.plot(score_gross['score'], m*score_gross['score']+b)





KeyError: 'profitability'